<a href="https://colab.research.google.com/github/rebeccaastaix/BASC0033/blob/main/Authors%26Publishers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import pandas as pd
from rapidfuzz import process, fuzz
import time
def get_author_google_fuzzy(title):
    url = "https://www.googleapis.com/books/v1/volumes"
    params = {"q": f"intitle:{title}"}
    r = requests.get(url, params=params)
    if r.status_code != 200:
        return None

    data = r.json()
    items = data.get("items", [])
    if not items:
        return None

    api_titles = []
    for item in items:
        volume = item.get("volumeInfo", {})
        api_title = volume.get("title")
        if api_title:
            api_titles.append(api_title)

    if not api_titles:
        return None

    best_match, score, idx = process.extractOne(title, api_titles, scorer=fuzz.token_sort_ratio)

    if score < 70:
        return None

    best_item = items[idx]
    volume_info = best_item.get("volumeInfo", {})

    authors = volume_info.get("authors")
    if authors:
        return authors[0]

    return None
df = pd.read_excel("Copy_Dataset(4086-records).xlsx")
for i, row in df[df['Name'].isna()].iterrows():
    title = row['Title'].strip()
    author = get_author_google_fuzzy(title)
    print(f"Fetching: {title} → {author}")
    df.at[i, 'Name'] = author
    time.sleep(0.2)
df.to_excel("full_spreadsheet_authors_filled.xlsx", index=False)
print("Finished! Saved as 'full_spreadsheet_authors_filled.xlsx'")


ModuleNotFoundError: No module named 'rapidfuzz'

In [ ]:
import requests
import pandas as pd
import time
def get_publisher_google(title):
    url = "https://www.googleapis.com/books/v1/volumes"
    params = {"q": f"intitle:{title}"}
    r = requests.get(url, params=params)
    if r.status_code != 200:
        return None
    data = r.json()
    try:
        return data["items"][0]["volumeInfo"].get("publisher")
    except:
        return None
df = pd.read_excel("Copy_Dataset(4086-records).xlsx")
for i, row in df[df['Publisher'].isna()].iterrows():
    title = row['Title'].strip()
    publisher = get_publisher_google(title)
    print(f"Fetching: {title} → {publisher}")
    df.at[i, 'Publisher'] = publisher
    time.sleep(0.2)
df.to_excel("full_spreadsheet_publishers_filled.xlsx", index=False)
print("Finished! Saved as 'full_spreadsheet_publishers_filled.xlsx'")
